In [1]:
import pandas as pd
import torch

from datetime import datetime, timedelta
from sqlalchemy import create_engine,exc,text
from transformers import pipeline
from tqdm import tqdm 

import warnings
warnings.filterwarnings("ignore")

In [2]:
# ---------------- CONFIGURAÇÕES ----------------
BATCH_SIZE = 32
MAX_TOKEN_LENGTH = 128  # Limite para o modelo
USE_GPU = torch.cuda.is_available()
DEVICE = 0 if USE_GPU else -1

In [3]:
print(f"Dispositivo para Hugging Face: {'GPU' if USE_GPU else 'CPU'}")

# ---------------- PIPELINE DE SENTIMENTO ----------------
print("Carregando modelo de análise de sentimento Hugging Face (pode demorar na primeira vez)...")
try:
    sentiment_pipeline = pipeline(
        "sentiment-analysis",
        model="pysentimiento/bertweet-pt-sentiment",
        device=DEVICE,
        truncation=True,
        top_k=None  # Isso fará com que ele mostre os scores de todas as classes
    )
    print("Modelo de análise de sentimento carregado com sucesso.")
except Exception as e:
    print(f"Erro ao carregar o modelo Hugging Face: {e}")
    sentiment_pipeline = None 

Dispositivo para Hugging Face: GPU
Carregando modelo de análise de sentimento Hugging Face (pode demorar na primeira vez)...


Device set to use cuda:0


Modelo de análise de sentimento carregado com sucesso.


In [4]:
# --- Configurações do MySQL
USUARIO = 'root'
SENHA = 'root'
HOST = 'localhost'
PORT = 3306
ORIGEM = 'vendas_ouvidoria'
DESTINO = 'vendas_analise'

In [5]:
# --- Nomes das colunas na sua tabela MySQL ---
# Ajuste estes nomes conforme as colunas reais da sua tabela
COLUNA_ID_CLIENTE = "id_cliente" # Coluna que contém o código do cliente
COLUNA_ID_MANIFESTACAO = "id" # Coluna que contém o ID único da manifestação (ex: _id do MongoDB, ou id do MySQL)
COLUNA_TEXTO_MANIFESTACAO = "descricao" # Coluna que contém o texto da manifestação

In [6]:
# --- Configurações do Modelo de Análise de Sentimento ---
# Modelo pré-treinado para sentimento em português (BERTimbau-based)
# Este modelo geralmente tem as labels: LABEL_0 (positivo), LABEL_1 (negativo), LABEL_2 (neutro)
SENTIMENT_MODEL_NAME = "finiteautomata/beto-sentiment-base"

# Mapeamento das labels do modelo para termos mais descritivos em português
SENTIMENT_LABELS_MAP = {
    "LABEL_0": "positivo",
    "LABEL_1": "negativo",
    "LABEL_2": "neutro"
}

In [7]:
# ---------------- FUNÇÃO PARA BUSCAR INTERAÇÕES ----------------
def get_sentimento():
    if sentiment_pipeline is None:
        return pd.DataFrame(columns=['id_cliente', 'id_interacao', 'sentiment_label']),'Sem pipeline'
    try:
        engineori = create_engine(f"mysql+pymysql://{USUARIO}:{SENHA}@{HOST}:{PORT}/{ORIGEM}")
    except exc.SQLAlchemyError as e:
        return pd.DataFrame(columns=['id_cliente', 'id_interacao', 'sentiment_label']),'Sen conexão com o banco'
    with engineori.connect() as connori:
        client = None 
        try:
            sql = 'select id_cliente, id as id_interacao, descricao from interacoes'
            df = pd.read_sql(sql,engineori)
            if df.empty:
                 return pd.DataFrame(columns=['id_cliente', 'id_interacao', 'sentiment_label']),'Sem dados'
   
            # --- Processamento de Sentimento em Lotes (BATCH) com barra de progresso ---           
            descriptions_list = df['descricao'].tolist()          
            sentiment_labels = []            
            batch_size = 64 # Ajuste conforme sua memória RAM/GPU.
            
            for i in tqdm(range(0, len(descriptions_list), batch_size), desc="Analisando Sentimento"):
                batch_descriptions = descriptions_list[i : i + batch_size]                
                batch_sentiment_results = sentiment_pipeline(batch_descriptions)               
                for results_for_text in batch_sentiment_results:
                    if results_for_text:
                        melhor_resultado = max(results_for_text, key=lambda x: x['score'])
                        label = melhor_resultado['label']
                    
                    sentiment_labels.append(label)
            
            df['sentiment_label'] = sentiment_labels
            df.drop('descricao',axis=1,inplace=True)
            return df,'OK'
        
        except Exception as e:
            return pd.DataFrame(columns=['id_cliente', 'id_interacao', 'sentiment_label']),e
        finally:
            if client:
                client.close()

In [8]:
df,mens = get_sentimento()

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset[00:11<46:12,  1.12s/it]
Analisando Sentimento: 100%|████████████████████████████████████████████████████████████████████| 2492/2492 [51:39<00:00,  1.24s/it]


In [9]:
df

,id_cliente,id_interacao,sentiment_label
0,28,1,NEG
1,1330,2,NEU
2,418,3,NEG
3,1599,4,NEG
4,3310,5,POS
...,...,...,...
159448,175653,159117,NEG
159449,191240,159118,NEU
159450,165848,159119,NEG
159451,92522,159120,NEG


In [10]:
mens

'OK'

In [13]:
try:
    enginedes = create_engine(f"mysql+pymysql://{USUARIO}:{SENHA}@{HOST}:{PORT}/{DESTINO}")
    with enginedes.connect() as conndes:
        print(f"Conexão com o banco '{DESTINO}' estabelecida com sucesso.")
except exc.SQLAlchemyError as e:
    print(f"Erro ao conectar ao banco '{DESTINO}': {e}")
    exit()

Conexão com o banco 'vendas_analise' estabelecida com sucesso.


In [15]:
try:
    df.to_sql('cliente_sentimento',if_exists='append',index=False,con=enginedes)
    print("Sentimentos carregado com sucesso.")
except exc.SQLAlchemyError as e:
        print(f"Erro ao carregar: {e}")

Sentimentos carregado com sucesso.
